### RAG란?
- LLM 성능을 높이는 기법 중 하나. 외부 데이터를 활용해서 LLM의 성능을 높이는 기법이다.


### 필요한 라이브러리 설치

In [1]:
pip install langchain langchain-openai langchain_community chromadb -q

Note: you may need to restart the kernel to use updated packages.


### API키 환경변수에 설정

In [9]:
import os

os.environ["OPENAI_API_KEY"] = "api_key"

### RAG 파이프라인 설계

Step 1 :: 데이터 불러오기 (Load Data)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

# 불러올 url 설정
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
# url 받을 준비하기
loader = WebBaseLoader(url)

# load()는 웹페이지를 읽고, 분석할 수 있음
docs = loader.load()

# url의 문서 총 합계
print(len(docs))
# 불러온 url 첫 번째 문서 내용 확인
print(len(docs[0].page_content))
# 첫 번째 문서 내용 중 5000번째 ~ 5999번째 텍스트 확인
print(docs[0].page_content[5000:6000])

Step 2 :: 텍스트 쪼개기 (Text Split)

In [7]:
# 텍스트 쪼갤 수 있는 클래스 가져오기
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1000, 200 단위는 '문자' (공백포함)
# 어떻게 쪼갤지 옵션 정해서 text_splitter에 저장
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
# split_documents()에 쪼갤 문서 넣기
splits = text_splitter.split_documents(docs)

# 1개 문서가 몇 개로 쪼개어졌나 확인
print(len(splits))
# 쪼개어진 문서 중 12번째 문서 확인
print(splits[11])

18
page_content='차후 공지가 불충분했다는 이의 제기를 피하려면, 위의 링크를 이용하여 공지하세요. 공지에 비중립적인 단어를 사용하는 등의 선전 행위는 피하세요.
토론이 끝났다면 선언과 함께 {{토론보존}} 틀을 이용하여 닫습니다. 총의 판단은 여타 토론과 마찬가지로  분쟁 해결 정책에서 갈음해 처리합니다. 토론을 통해 정책이나 지침 채택 여부를 논의하며, 이 과정에서 제안 문서가 크게 수정될 수도 있습니다. 토론 중 제안을 정식 정책/지침으로 채택하자는 합의로 모이고 나서 2주 (정확히 14일. 이후 내용은 모두 같습니다) 간 제안을 대폭 수정해야 하는 변경안 제시나 명확한 근거가 존재하는 반대가 나오지 않는다면 정책이나 지침으로 정식으로 채택됩니다. 반대로 토론자들 사이에서 채택을 거부한다는 합의가 모아져서 2주간 명확한 근거가 존재하는 반대 의견이 나오지 않는다면 채택안 거부 총의가 모아졌다고 보아 기각됩니다. 주요한 총의 판단 기준은 다음과 같습니다.' metadata={'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8', 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전', 'language': 'ko'}


Step 3 :: 쪼갠 텍스트 백터화 한 후(임베딩) 데이터 정리하기(인덱싱)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 백터화 할 문서 지정 후 백터값 vectorstore에 지정
vectorstore = Chroma.from_documents(documents = splits, embedding=OpenAIEmbeddings())

# similarity_search() 이용해서 백터값에 질문하기
docs = vectorstore.similarity_search("격화 과정에 대해 설명해 주세요.")

# 질문과 유사한 문서 개수 총합
print(len(docs))
# 유사한 문서 중 첫 번째 문서의 내용 확인
print(docs[0].page_content)

Step 4 :: 정리한 데이터에서 사용자가 원하는 내용 검색(R) 후 생성(G)

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

# 템플릿으로 프롬프트 설정
prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 설정
# 답변 일관성을 위해 temperature=0 설정
model = ChatOpenAI(model = 'gpt-4o-mini', temperature=0)

# 백터화 된 데이터에서 검색하기
retriever = vectorstore.as_retriever()

# 질문에 따라 쪼개어진 문서 한 텍스트로 합치기
def combine_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# 1번째 체인(검색, R): 검색 -> 합쳐진 백터화 문서, RunnablePassthrough()
# -> 실행 순서 : RunnablePassthrough - 검색 - 합쳐진 백터화 문서
# 2번째 체인 : 프롬프트 설정
# 3번째 체인 : 모델 설정
# 4번째 체인(생성, G)  
rag_chain = (
    {'context': retriever | combine_docs, 'question' : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# invoke에 작성한 내용이 RunnablePassthrough()로 이동
rag_chain.invoke("격하 과정에 대해서 설명해주세요.")

'격하 과정은 특정 정책이나 지침이 더 이상 쓸모없거나 중복되는 경우, 이를 지침이나 다른 형태로 변경하는 절차입니다. 이 과정은 채택 과정과 유사하게 진행됩니다. 일반적으로 토론 문서 내에서 논의가 시작되며, 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 충분한 논의가 이루어진 후, 제3의 편집자가 토론을 종료하고 상태 변경에 대한 총의를 판단합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않음을 알립니다. 소수의 공동체 인원만 지지하는 비공식 문서는 주된 작성자의 사용자 이름공간으로 이동될 수 있으며, 이러한 논의는 해당 문서의 토론란에서 이루어지거나 위키백과:의견 요청을 통해 처리될 수 있습니다.'